University of Central Missouri
Department of Computer Science & Cybersecurity

CS5760 Natural Language Processing
Fall 2025




Homework 1.



Student name:SANTHOSH REDDY KISTIPATI

Q1. Regex
Task: Write a regex to find
U.S. ZIP codes (disjunction + token boundaries)
Match 12345 or 12345-6789 or 12345 6789 (hyphen or space allowed for the +4 part). Make sure you only match whole tokens (not inside longer strings).

ANs:\b\d{5}(?:[ -]\d{4})?\b

2) Negation in disjunction (word start rules)
Find all words that do not start with a capital letter. Words may include internal apostrophes/hyphens like don’t, state-of-the-art.

ANs: \b(?![A-Z])[a-z]+(?:['’-][a-z]+)*\b


3) Convenient aliases (numbers, a bit richer)
Extract all numbers that may have:
optional sign (+/-),
optional thousands separators (commas),
optional decimal part,
optional scientific notation (e.g., 1.23e-4).

Ans: \b[+-]?(?:(?:\d{1,3}(?:,\d{3})+|\d+)(?:\.\d+)?|\.\d+)(?:[eE][+-]?\d+)?\b


4) More disjunction (spelling variants)
Match any spelling of “email”: email, e-mail, or e mail. Accept either a space or a hyphen (including en‐dash –) between e and mail, and be case-insensitive.

Ans: (?i)\bE(?:[- ]?)mail\b
       With en- dash : (?i)\bE(?:\s|[-\u2013])?mail\b


5 )Wildcards, optionality, repetition (with punctuation)
Match the interjection go, goo, gooo, … (one or more o), as a word, and allow an optional trailing punctuation mark ! . , ? (e.g., gooo!).

ANS: \bgo+\b[!.,?]?


6) Anchors (line/sentence end with quotes)

Match lines that end with a question mark possibly followed only by closing quotes/brackets like ")”’] and spaces.

Ans :(?m)^[^\n]*\?[)"\]\u201D\u2019\s]*$



In [ ]:
Q2. Programming Question:
1. Tokenize a paragraph
Take a short paragraph (3–4 sentences) in your language (e.g., from news, a story, or social media).

Ans: "Artificial Intelligence is growing rapidly. It's changing industries worldwide! However, challenges remain, including ethics and bias."
Do naïve space-based tokenization.
["Artificial", "Intelligence", "is", "growing", "rapidly.", "It's", "changing", "industries", "worldwide!", "However,", "challenges", "remain,", "including", "ethics", "and", "bias."]


Manually correct the tokens by handling punctuation, suffixes, and clitics.
Submit both versions and highlight differences.

["Artificial", "Intelligence", "is", "growing", "rapidly", ".", "It", "'s", "changing", "industries", "worldwide", "!", "However", ",", "challenges", "remain", ",", "including", "ethics", "and", "bias", "."]


2. Compare with a Tool
Run the paragraph through an NLP tool that supports your language (e.g., NLTK, spaCy, or any open-source tokenizer if available).
Compare tool output vs. your manual tokens.
Which tokens differ? Why?

ANs: Native tokenization kept punctuation attached (e.g., "rapidly.", "worldwide!") and did not split "It's".
Manual and NLTK both separated punctuation and clitics.
Conclusion: NLTK matches the manual correction, while naïve space-splitting fails.



3. Multiword Expressions (MWEs)
Identify at least 3 multiword expressions (MWEs) in your language. Example:
Place names, idioms, or common fixed phrases.
Explain why they should be treated as single tokens.

Ans : New York City — proper noun, should be one token.
machine learning — technical phrase, used as one concept.
kick the bucket — idiom, meaning cannot be derived from individual words.



4. Reflection (5–6 sentences)
What was the hardest part of tokenization in your language?

Ans : The hardest part of tokenization was handling punctuation and clitics (e.g., splitting "It's" into "It" + "'s").

How does it compare with tokenization in English?

ans: NLTK made this easier by automatically handling contractions and punctuation. Compared to English, languages with richer morphology or without spaces are even more challenging.


Do punctuation, morphology, and MWEs make tokenization more difficult?

Ans : Multiword expressions add complexity since their meaning goes beyond word-by-word tokenization.



In [3]:
# 2 question 2.2 program
# Q2. Compare with a Tool (NLTK example)

# Install NLTK if needed
# !pip install nltk

import nltk
# Download the 'punkt' resource if needed. The error message suggests downloading 'punkt_tab',
# but 'punkt' is the commonly used resource for tokenization.
nltk.download('punkt')
nltk.download('punkt_tab') # Added to download the specific resource

from nltk.tokenize import word_tokenize
import re

# Paragraph
text = "Artificial Intelligence is growing rapidly. It's changing industries worldwide! However, challenges remain, including ethics and bias."

# Manual tokenization (from Q1)
def manual_tokenize(text):
    text = text.replace("’", "'")
    text = re.sub(r"(\w)('ll|'re|'ve|n't|'s|'d|'m)\b", r"\1 \2", text)
    text = re.sub(r"([.,!?;:()])", r" \1 ", text)
    tokens = re.split(r"\s+", text.strip())
    return [t for t in tokens if t]

manual_tokens = manual_tokenize(text)

# NLTK tool tokenization
nltk_tokens = word_tokenize(text)

print("Manual tokens:", manual_tokens)
print("NLTK tokens:", nltk_tokens)

# Compare
differences = [(m, n) for m, n in zip(manual_tokens, nltk_tokens) if m != n]
print("\nDifferences:", differences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Manual tokens: ['Artificial', 'Intelligence', 'is', 'growing', 'rapidly', '.', 'It', "'s", 'changing', 'industries', 'worldwide', '!', 'However', ',', 'challenges', 'remain', ',', 'including', 'ethics', 'and', 'bias', '.']
NLTK tokens: ['Artificial', 'Intelligence', 'is', 'growing', 'rapidly', '.', 'It', "'s", 'changing', 'industries', 'worldwide', '!', 'However', ',', 'challenges', 'remain', ',', 'including', 'ethics', 'and', 'bias', '.']

Differences: []


In [4]:
# ==============================================
# Q3 — BYTE PAIR ENCODING (BPE): FULL SOLUTION
# With PRINT STATEMENTS for every question/subpart
# ==============================================

from collections import Counter
import re

# ---------- Common helpers ----------

def word_to_symbols(word):
    """Convert a word into characters + end-of-word marker '_'."""
    return list(word) + ['_']

def apply_one_merge_to_word(word, a, b):
    """Apply a single merge (a,b)->ab to one word represented as symbols."""
    syms = word_to_symbols(word)
    out, i = [], 0
    while i < len(syms):
        if i + 1 < len(syms) and syms[i] == a and syms[i+1] == b:
            out.append(a + b)
            i += 2
        else:
            out.append(syms[i])
            i += 1
    return out

def apply_one_merge_to_corpus(words, a, b):
    """Apply a single merge to all words; return list of space-separated symbol strings."""
    return [' '.join(apply_one_merge_to_word(w, a, b)) for w in words]

def bigram_counts_within_words(words):
    """Count bigrams inside words (including final (last_char, '_'))."""
    c = Counter()
    for w in words:
        syms = word_to_symbols(w)
        for i in range(len(syms) - 1):
            c[(syms[i], syms[i+1])] += 1
    return c

def words_to_symbol_lists(merged_strings):
    """['n e w er _', ...] -> [['n','e','w','er','_'], ...]."""
    return [s.split() for s in merged_strings]

def bigram_counts_from_merged(merged_strings):
    """Recount bigrams after merges when words are stored as strings of symbols."""
    c = Counter()
    for syms in words_to_symbol_lists(merged_strings):
        for i in range(len(syms) - 1):
            c[(syms[i], syms[i+1])] += 1
    return c

# ==========================================================
# 3.1 — Manual BPE on a toy corpus
# ==========================================================
print("\n====================")
print("Q3.1 — Manual BPE on a Toy Corpus")
print("====================")

tokens_31 = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new".split()

# 3.1.1
print("\n3.1.1 Add end-of-word marker '_' and write the initial vocabulary (characters + '_'):")
initial_vocab_31 = set()
for w in tokens_31:
    initial_vocab_31.update(word_to_symbols(w))
print("Initial symbol vocabulary:", sorted(initial_vocab_31))

# 3.1.2
print("\n3.1.2 Compute bigram counts and perform the first three merges by hand:")

# Step 1
bigrams0 = bigram_counts_within_words(tokens_31)
(a1, b1), f1 = bigrams0.most_common(1)[0]
m1 = apply_one_merge_to_corpus(tokens_31, a1, b1)
print("\n  Step 1 — Most frequent pair:", (a1, b1), "freq=", f1, "→ merge", a1 + b1)
print("  Step 1 — Updated corpus snippet (first 8 tokens shown):")
print("   ", "  |  ".join(m1[:8]))
vocab1 = set(' '.join(m1).split())
print("  Step 1 — New token:", a1 + b1, " | Updated vocabulary size:", len(vocab1))

# Step 2
bigrams1 = bigram_counts_from_merged(m1)
(a2, b2), f2 = bigrams1.most_common(1)[0]
m2_part = apply_one_merge_to_corpus(tokens_31, a1, b1)                # re-apply step 1 cleanly
m2 = apply_one_merge_to_corpus([''.join(s.split()) for s in m2_part],  # compact then merge step 2
                               a2, b2)
print("\n  Step 2 — Most frequent pair:", (a2, b2), "freq=", f2, "→ merge", a2 + b2)
print("  Step 2 — Updated corpus snippet (first 8 tokens shown):")
print("   ", "  |  ".join(m2[:8]))
vocab2 = set(' '.join(m2).split())
print("  Step 2 — New token:", a2 + b2, " | Updated vocabulary size:", len(vocab2))

# Step 3
bigrams2 = bigram_counts_from_merged(m2)
(a3, b3), f3 = bigrams2.most_common(1)[0]
m3_part = apply_one_merge_to_corpus(tokens_31, a1, b1)
m3_part = apply_one_merge_to_corpus([''.join(s.split()) for s in m3_part], a2, b2)
m3 = apply_one_merge_to_corpus([''.join(s.split()) for s in m3_part], a3, b3)
print("\n  Step 3 — Most frequent pair:", (a3, b3), "freq=", f3, "→ merge", a3 + b3)
print("  Step 3 — Updated corpus snippet (first 8 tokens shown):")
print("   ", "  |  ".join(m3[:8]))
vocab3 = set(' '.join(m3).split())
print("  Step 3 — New token:", a3 + b3, " | Updated vocabulary size:", len(vocab3))

print("\n3.1.3 After each merge, the new token and updated vocabulary size were printed above.")

# ==========================================================
# 3.2 — Code a mini-BPE learner
# ==========================================================
print("\n====================")
print("Q3.2 — Code a Mini-BPE Learner")
print("====================")

def get_vocab_from_words(words):
    """Map tuple(symbols) -> count for the corpus vocabulary."""
    return Counter(tuple(word_to_symbols(w)) for w in words)

def get_pair_counts(vocab):
    """Count adjacent symbol pairs across the vocabulary with frequencies."""
    pc = Counter()
    for syms, freq in vocab.items():
        for i in range(len(syms) - 1):
            pc[(syms[i], syms[i+1])] += freq
    return pc

def merge_vocab_once(vocab, pair):
    """Merge a pair (a,b)→ab across vocabulary (regex on space-joined symbols)."""
    a, b = pair
    pat = re.compile(r'(?<!\S)'+re.escape(a)+r'\s+'+re.escape(b)+r'(?!\S)')
    new_vocab = Counter()
    for syms, freq in vocab.items():
        s = ' '.join(syms)
        s = pat.sub(a + b, s)
        new_syms = tuple(s.split())
        new_vocab[new_syms] += freq
    return new_vocab

def learn_bpe(vocab, steps):
    merges = []
    for _ in range(steps):
        pc = get_pair_counts(vocab)
        if not pc:
            break
        (a, b), f = pc.most_common(1)[0]
        merges.append(((a, b), f))
        vocab = merge_vocab_once(vocab, (a, b))
    return merges, vocab

def apply_bpe_to_word(word, merges):
    syms = word_to_symbols(word)
    for (a, b), _ in merges:
        i, out = 0, []
        while i < len(syms):
            if i + 1 < len(syms) and syms[i] == a and syms[i+1] == b:
                out.append(a + b)
                i += 2
            else:
                out.append(syms[i])
                i += 1
        syms = out
    return syms

toy_words = ["low"]*5 + ["lowest"]*2 + ["newer"]*6 + ["wider"]*3 + ["new"]*2

# 3.2.1
print("\n3.2.1 Learn BPE merges for the toy corpus and print the top pair at each step:")
vocab_32 = get_vocab_from_words(toy_words)
merges_32, _ = learn_bpe(vocab_32, steps=10)
for i, ((a, b), f) in enumerate(merges_32, 1):
    print(f"  Step {i}: {a}+{b} (freq={f})")

# 3.2.2
print("\n3.2.2 Segment the words: new, newer, lowest, widest, newestest (show tokens with '_'):")
for w in ["new", "newer", "lowest", "widest", "newestest"]:
    print(f"  {w} -> {apply_bpe_to_word(w, merges_32)}")

# 3.2.3
print("\n3.2.3 Reflection (concise, printed):")
print("  Subwords solve OOV by composing unseen words from known pieces, avoiding zero probabilities.")
print("  Example morpheme alignment: 'er_' often appears as a comparative/agentive suffix at word end in English.")

# ==========================================================
# 3.3 — Train BPE on a short paragraph (English)
# ==========================================================
print("\n====================")
print("Q3.3 — Train BPE on a Short Paragraph (English)")
print("====================")

# 3.3.1
print("\n3.3.1 Train BPE on a 6-sentence paragraph with end-of-word '_' and learn ≥30 merges:")
paragraph_33 = (
    "Artificial intelligence is transforming many industries. "
    "Machine learning models improve with data, but ethics and transparency matter. "
    "Natural language processing enables powerful applications across domains. "
    "However, fairness, bias mitigation, and robust evaluation remain essential. "
    "Developers and researchers iterate quickly in production environments. "
    "Clear documentation and responsible deployment build long-term trust."
).lower()
words_33 = re.findall(r"[a-z]+", paragraph_33)
vocab_33 = get_vocab_from_words(words_33)
merges_33, final_vocab_33 = learn_bpe(vocab_33, steps=30)
print("  Learned", len(merges_33), "merges.")

# 3.3.2
print("\n3.3.2 Five most frequent merges (first five learned) and five longest subword tokens:")
for (a, b), f in merges_33[:5]:
    print(f"  Merge: {a}+{b} (freq={f})")

inventory_33 = set()
for syms in final_vocab_33:
    for s in syms:
        if s:
            inventory_33.add(s)
longest_5 = sorted(inventory_33, key=len, reverse=True)[:5]
print("  Five longest learned subword tokens:", longest_5)

# 3.3.3
print("\n3.3.3 Segment 5 words from the paragraph (include rare/derived if possible):")
for w in ["artificial", "transparency", "developers", "processing", "trust"]:
    print(f"  {w} -> {apply_bpe_to_word(w, merges_33)}")

# 3.3.4
print("\n3.3.4 Brief reflection (concise, printed):")
print("  Learned subwords capture frequent stems and affixes (e.g., 'ing_', 'tion', 'ation') and common clusters.")
print("  Pros: reduces OOV by composing rare/novel forms; keeps vocabulary compact; helps with morphology.")
print("  Cons: merges are frequency-driven and may split across morpheme boundaries; learned merges are domain-sensitive.")



Q3.1 — Manual BPE on a Toy Corpus

3.1.1 Add end-of-word marker '_' and write the initial vocabulary (characters + '_'):
Initial symbol vocabulary: ['_', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']

3.1.2 Compute bigram counts and perform the first three merges by hand:

  Step 1 — Most frequent pair: ('e', 'r') freq= 9 → merge er
  Step 1 — Updated corpus snippet (first 8 tokens shown):
    l o w _  |  l o w _  |  l o w _  |  l o w _  |  l o w _  |  l o w e s t _  |  l o w e s t _  |  n e w er _
  Step 1 — New token: er  | Updated vocabulary size: 11

  Step 2 — Most frequent pair: ('er', '_') freq= 9 → merge er_
  Step 2 — Updated corpus snippet (first 8 tokens shown):
    l o w _ _  |  l o w _ _  |  l o w _ _  |  l o w _ _  |  l o w _ _  |  l o w e s t _ _  |  l o w e s t _ _  |  n e w e r _ _
  Step 2 — New token: er_  | Updated vocabulary size: 11

  Step 3 — Most frequent pair: ('_', '_') freq= 18 → merge __
  Step 3 — Updated corpus snippet (first 8 tokens shown):
    l o

In [ ]:
#Q4. Word Pair:
Sunday → Saturday
Tasks:
Find the minimum edit distance between Sunday and Saturday under both models:
Model A (Sub = 1, Ins = 1, Del = 1)
Model B (Sub = 2, Ins = 1, Del = 1)

1) Minimum edit distance
Model A (Sub=1, Ins=1, Del=1): Distance = 3
Model B (Sub=2, Ins=1, Del=1): Distance = 4


#2 . One valid edit sequence
Model A (cost 3)
Insert a after S → Saunday
Insert t after Sa → Satunday
Substitute n → r → Saturday
Model B (cost 4)
Insert a after S → Saunday
Insert t after Sa → Satunday
Delete n → Satuday
Insert r before d → Saturday

3. Reflection (4–5 sentences)
The two models gave different distances: Model A = 3, Model B = 4. In Model A, substitution was cheap and useful, so it was chosen for the mismatch.
In Model B, substitution was expensive, so the algorithm preferred a delete + insert instead. The choice of model matters: for spell-checking, substitutions should be cheap since typos often involve wrong letters.
For DNA alignment, substitutions (mutations) may be more costly or biologically significant, so making substitutions expensive produces more realistic alignments.